In [ ]:
# CPE453 Program 4 - TinyFS and Disk emulator
# Damien Trunkey, Danveer Cheema, Sereen Benchohra

In [137]:
import sys
import os
import pickle
from libDisk import *
# global variables
BLOCKSIZE = 256
MAGICNUMBER = 0x5A
fileSystems = []
dynamicResourceTable = []
fileDesc = 0

In [138]:
# This function opens a regular UNIX file and designates the first nBytes of it as space for the emulated disk. 
# nBytes should be a number that is evenly divisible by the block size. 
# If nBytes > 0 and there is already a file by the given filename, that disk is resized to nBytes, and that file’s contents may be overwritten. 
# If nBytes is 0, an existing disk is opened, and should not be overwritten. There is no requirement to maintain integrity of any content beyond nBytes. 
# Errors must be returned for any other failures, as defined by your own error code system.
def openDisk(filename, nBytes):
    fd = 0
    # File doesnt exist. Return error
    if nBytes < 0:
        return -1
    # do nothing. file is already open
    if nBytes == 0:
        return
    try:
        # |os.O_EXCL
        fd = os.open(filename, os.O_RDWR|os.O_CREAT|os.O_TRUNC)
    except:
        return -1
    return fd

In [139]:
# readBlock() reads an entire block of BLOCKSIZE bytes from the open disk (identified by ‘disk’) and copies the result into a local buffer (must be at least of BLOCKSIZE bytes). 
# The bNum is a logical block number, which must be translated into a byte offset within the disk. 
# The translation from logical to physical block is straightforward: bNum=0 is the very first byte of the file. 
# bNum=1 is BLOCKSIZE bytes into the disk, bNum=n is n*BLOCKSIZE bytes into the disk. 
# On success, it returns 0. Errors must be returned if ‘disk’ is not available (i.e. hasn’t been opened) or for any other failures, as defined by your own error code system.

def readBlock(disk, bNum, block):
    offset = bNum * BLOCKSIZE
    try:
        os.lseek(disk, offset, 0)
    except:
        print("File either does not exist or hasnt been opened")
        return -1
    block[0] = os.read(disk, BLOCKSIZE)
    return 0

In [140]:
# writeBlock() takes disk number ‘disk’ and logical block number ‘bNum’ and writes the content of the buffer ‘block’ to that location. 
# BLOCKSIZE bytes will be written from ‘block’ regardless of its actual size. The disk must be open. Just as in readBlock(), 
# writeBlock() must translate the logical block bNum to the correct byte position in the file. 
# On success, it returns 0. Errors must be returned if ‘disk’ is not available (i.e. hasn’t been opened) 
# or for any other failures, as defined by your own error code system.

def writeBlock(disk, bNum, block):
    offset = bNum * BLOCKSIZE
    try:
        os.lseek(disk, offset, 0)
    except:
        print("File either does not exist or hasnt been opened")
        return -1
    os.write(disk, block)
    return 0

In [141]:
# closeDisk() takes a disk number ‘disk’ and makes the disk closed to further I/O; i.e. any subsequent reads or writes to a closed disk should return an error. 
# Closing a disk should also close the underlying file, committing any writes being buffered by the real OS.

def closeDisk(disk):
    try:
        os.close(disk)
    except:
        print("File either does not exist or hasnt been opened")
        return -1

In [142]:
filename = "test.txt"
bnum = 0
fd = openDisk(filename, 256)
writeB = str.encode("this is a test to see if this works")
readB = [None]
writeBlock(fd, 0, writeB)
readBlock(fd, bnum, readB)
print(readB[0])

b'this is a test to see if this works'


In [143]:
class superBlock():
    magicNumber = 0
    freeBlocks = [None] * BLOCKSIZE
    def __init__(self, magicNumber, freeBlocks):
        self.magicNumber = magicNumber
        self.freeBlocks = freeBlocks

In [144]:
class Inode():
    name = None
    stats = None
    def __init__(self, name, stats):
        self.name = name
        self.stats = stats

In [145]:
class dynamicResourceEntry():
    filePointer = 0
    blockIndex = 0
    fd = 0
    def __init__(self, filePointer, blockIndex, fd):
        self.filePointer = filePointer
        self.blockIndex = blockIndex
        self.fd = fd

In [146]:
class fileSystem():
    fileName = None
    fd = 0
    mounted = None
    superBlock = None
    def __init__(self, fileName, fd, mounted, superBlock):
        self.fileName = fileName
        self.fd = fd
        self.mounted = mounted
        self.superBlock = superBlock

In [147]:
# Makes an empty TinyFS file system of size nBytes on the file specified by ‘filename’. 
# This function should use the emulated disk library to open the specified file, and upon success, format the file to be mountable. 
# This includes initializing all data to 0x00, setting magic numbers, initializing and writing the superblock and other metadata, etc. 
# Must return a specified success/error code.

# pickle.dumps(<object>) turns a python object into a byte array so we can write it to a file
def tfs_mkfs(filename, nBytes):
    fd = openDisk(filename, nBytes)
    if fd < 0:
        print("File open error")
        return -1
    
    blocks = [None] * int(nBytes/BLOCKSIZE)
    sBlock = superBlock(MAGICNUMBER, blocks)
    if writeBlock(fd, 0, pickle.dumps(sBlock)) < 0:
        print("write error")
        return -1
    
    stats = os.fstat(fd)
    rootInode = Inode("/", stats)
    if writeBlock(fd, 1, pickle.dumps(rootInode)) < 0:
        print("write error")
        return -1
    
    fs = fileSystem(filename, fd, 0, sBlock)
    
    fileSystems.append(fs)
    return 0


In [148]:
tfs_mkfs("test.txt", 256)
f = open('test.txt', 'rb')
print(f.readlines())

[b'\x80\x04\x95D\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\r\n', b'superBlock\x94\x93\x94)\x81\x94}\x94(\x8c\x0bmagicNumber\x94KZ\x8c\r\n', b"freeBlocks\x94]\x94Naub.\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x04\x95(\x01\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x05Inode\x94\x93\x94)\x81\x94}\x94(\x8c\x04name\x94\x8c\x0

In [149]:
# tfs_mount(char *filename) “mounts” a TinyFS file system located within ‘filename’. 
# tfs_unmount(void) “unmounts” the currently mounted file system. 
# As part of the mount operation, tfs_mount should verify the file system is the correct type. 
# Only one file system may be mounted at a time. 
# Use tfs_unmount to cleanly unmount the currently mounted file system. Must return a specified success/error code

def tfs_mount(filename):   
    tfs_unmount()
    for fs in fileSystems:
        if fs.fileName == filename:
            fs.mounted = 1
    return 0

In [150]:
tfs_mount("test.txt")
print(fileSystems[0].fileName, "Mounted (1 = yes, 0 = no)", fileSystems[0].mounted)

test.txt Mounted (1 = yes, 0 = no) 1


In [106]:
def tfs_unmount():
    for file in fileSystems:
        # unmount
        if file.mounted == 1:
            file.mounted = 0
    return 0

In [107]:
tfs_unmount()
print(fileSystems[0].fileName, "Mounted (1 = yes, 0 = no)", fileSystems[0].mounted)

test.txt Mounted (1 = yes, 0 = no) 0


In [151]:
# Opens a file for reading and writing on the currently mounted file system. 
# Creates a dynamic resource table entry for the file (the structure that tracks open files, the internal file pointer, etc.), 
# and returns a file descriptor (integer) that can be used to reference this file while the filesystem is mounted.
def tfs_open(name):
    global fileDesc
    currFileSystem = None
    fileIndex = 0
    for fs in fileSystems:
        if fs.mounted == 1:
            currFileSystem = fs
    for i in range(len(currFileSystem.superBlock.freeBlocks)):
        if currFileSystem.superBlock.freeBlocks[i] == None:
            fileIndex = i
    
    stats = os.fstat(currFileSystem.fd)
    inode = Inode(name, stats)
    currFileSystem.superBlock.freeBlocks[fileIndex] = inode
    if writeBlock(currFileSystem.fd, fileIndex, pickle.dumps(currFileSystem.superBlock)) < 0:
        print("write error")
        return -1
    fileDesc = fileDesc + 1
    dre = dynamicResourceEntry(0, fileIndex, fileDesc)
    dynamicResourceTable.append(dre)
    return fileDesc      

In [152]:
fd = tfs_open("im_a_file")
f = open('test.txt', 'rb')
print(f.readlines())
print(fd)

[b'\x80\x04\x95j\x01\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\r\n', b'superBlock\x94\x93\x94)\x81\x94}\x94(\x8c\x0bmagicNumber\x94KZ\x8c\r\n', b"freeBlocks\x94]\x94h\x00\x8c\x05Inode\x94\x93\x94)\x81\x94}\x94(\x8c\x04name\x94\x8c\tim_a_file\x94\x8c\x05stats\x94\x8c\x02os\x94\x8c\x0bstat_result\x94\x93\x94(M\xb6\x81\x8a\x08J\x05\x03\x00\x00\x00\x84\x00J\x96\xce\xeaNK\x01K\x00K\x00M3\x02J'\xa0+bJ'\xa0+bJ\xedC(bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe8\t\xd7\xc5&\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\t\xd7cP\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x08\xe4\x02\xf8\xd6\x9di\xdb\x16\x8c\x0bst_mtime_ns\x94\x8a\x08T\xe4\x9c\xd6\x9di\xdb\x16\x8c\x0bst_ctime_ns\x94\x8a\x08\x18\x08-qT\xa1\xda\x16\x8c\x12st_file_attributes\x94K \x8c\x0est_reparse_tag\x94K\x00u\x86\x94R\x94ubaub.bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe8\t\xd7R\xc5\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\t\xd7R\xc5\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x080|\x8d\x

In [113]:
# Closes the file and removes dynamic resource table entry
def tfs_close(FD):
    for entry in dynamicResourceTable:
        if entry.fd == FD:
            dynamicResourceTable.remove(entry)

In [114]:
print(len(dynamicResourceTable))
tfs_close(fd)
print(len(dynamicResourceTable))

1
0


In [153]:
# Writes buffer ‘buffer’ of size ‘size’, which represents an entire file’s contents, 
# to the file described by ‘FD’. 
# Sets the file pointer to 0 (the start of file) when done. 
# Returns success/error codes.
def tfs_write(FD, buffer, size):
    for entry in dynamicResourceTable:
        if entry.fd == FD:
            blockIndex = entry.blockIndex
    for fs in fileSystems:
        if fs.mounted == 1:
            fs.superBlock.freeBlocks.append(str.encode(buffer))
            if writeBlock(fs.fd, blockIndex, pickle.dumps(fs.superBlock.freeBlocks)) < 0:
                print("write error")
                return -1

In [154]:
fd = tfs_open("im_a_file")
tfs_write(fd, "hello this is a test", 256)
f = open('test.txt', 'rb')
print(f.readlines())

[b'\x80\x04\x95L\x01\x00\x00\x00\x00\x00\x00]\x94(\x8c\x08__main__\x94\x8c\x05Inode\x94\x93\x94)\x81\x94}\x94(\x8c\x04name\x94\x8c\tim_a_file\x94\x8c\x05stats\x94\x8c\x02os\x94\x8c\x0bstat_result\x94\x93\x94(M\xb6\x81\x8a\x08J\x05\x03\x00\x00\x00\x84\x00J\x96\xce\xeaNK\x01K\x00K\x00M3\x02J,\xa0+bJ,\xa0+bJ\xedC(bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe8\x0b\x04\xf9\\\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\x0b\x04\x86\xfb\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x080\xa0|\xef\x9ei\xdb\x16\x8c\x0bst_mtime_ns\x94\x8a\x08\xa8\x1a\x12\xef\x9ei\xdb\x16\x8c\x0bst_ctime_ns\x94\x8a\x08\x18\x08-qT\xa1\xda\x16\x8c\x12st_file_attributes\x94K \x8c\x0est_reparse_tag\x94K\x00u\x86\x94R\x94ubC\x14hello this is a test\x94e.K \x8c\x0est_reparse_tag\x94K\x00u\x86\x94R\x94ubaub.bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe8\t\xd7R\xc5\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\t\xd7R\xc5\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x080|\x8d\xd6\x9di\xdb\x16\x8

In [117]:
# deletes a file and marks its blocks as free on disk
def tfs_delete(FD):
    for entry in dynamicResourceTable:
        if entry.fd == FD:
             blockIndex = entry.blockIndex
    for fs in fileSystems:
        if fs.mounted == 1:
            fs.superBlock.freeBlocks[blockIndex] = None
            if writeBlock(fs.fd, blockIndex, pickle.dumps(fs.superBlock.freeBlocks)) < 0:
                print("write error")
                return -1

In [155]:
fd = tfs_open("im_a_file")
tfs_delete(fd)
#print(fileSystems[0].superBlock.freeBlocks[])
f = open('test.txt', 'rb')
print(f.readlines())

[b'\x80\x04\x95\x1d\x00\x00\x00\x00\x00\x00\x00]\x94(NC\x14hello this is a test\x94e.\x94}\x94(\x8c\x0bmagicNumber\x94KZ\x8c\r\n', b'freeBlocks\x94]\x94(h\x00\x8c\x05Inode\x94\x93\x94)\x81\x94}\x94(\x8c\x04name\x94\x8c\tim_a_file\x94\x8c\x05stats\x94\x8c\x02os\x94\x8c\x0bstat_result\x94\x93\x94(M\xb6\x81\x8a\x08J\x05\x03\x00\x00\x00\x84\x00J\x96\xce\xeaNK\x01K\x00K\x00M3\x02J7\xa0+bJ7\xa0+bJ\xedC(bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe8\r\xe1\xc3\x12\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\r\xe1P\xa9\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x08\\\xe2\xf2\x99\xa1i\xdb\x16\x8c\x0bst_mtime_ns\x94\x8a\x08hU\x88\x99\xa1i\xdb\x16\x8c\x0bst_ctime_ns\x94\x8a\x08\x18\x08-qT\xa1\xda\x16\x8c\x12st_file_attributes\x94K \x8c\x0est_reparse_tag\x94K\x00u\x86\x94R\x94ubC\x14hello this is a test\x94eub.R\xc5\x8c\x08st_mtime\x94GA\xd8\x8a\xe8\t\xd7R\xc5\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x080|\x8d\xd6\x9di\xdb\x16\x8c\x0bst_mtime_ns\x94\x

In [ ]:
# reads one byte from the file and copies it to ‘buffer’, 
# using the current file pointer location and incrementing it by one upon success. 
# If the file pointer is already at the end of the file then tfs_readByte() 
# should return an error and not increment the file pointer
def tfs_readByte(FD, buffer):
    for entry in dynamicResourceTable:
        if entry.fd == FD:
            blockIndex = entry.blockIndex
            filePtr = entry.filePointer
            entry.filePointer += 1
    # get current mounted file system
    for file in fileSystems:
        # found mounted file system now we get a single byte and write to buffer
        if file.mounted == 1:
            # readBlock(FD, blockIndex, buffer) 
            buffer[0] = file.superBlock.freeBlocks[blockIndex+1][filePtr]

In [120]:
fd = tfs_open("im_a_file")
buffer = [None]
tfs_readByte(fd, buffer)
print(chr(buffer[0]))

h


In [121]:
# change the file pointer location to offset (absolute). Returns success/error codes
def tfs_seek(FD, offset):
    for entry in dynamicResourceTable:
        if entry.fd == FD:
            entry.filePointer = offset

In [122]:
fd = tfs_open("im_a_file")
buffer = [None]
tfs_seek(fd, 8)
tfs_readByte(fd, buffer)
print(chr(buffer[0]))

i


In [123]:
def tfs_rename(filename, fd):
    for entry in dynamicResourceTable:
        if entry.fd == fd:
            fileIndex = entry.blockIndex
    for file in fileSystems:
        # found mounted file system now we get a single byte and write to buffer
        if file.mounted == 1:
            print("here")
            # readBlock(FD, blockIndex, buffer) 
            inode = file.superBlock.freeBlocks[0]
            inode.name = filename
            file.superBlock.freeBlocks[0] = inode
            writeBlock(file.fd, fileIndex, pickle.dumps(file.superBlock))

In [124]:
fd = tfs_open("im_a_file")
#print(fileSystems[0].superBlock.freeBlocks[])
f = open('test.txt', 'rb')
print(f.readlines())
tfs_rename("not_a_file", fd)
f = open('test.txt', 'rb')
print(f.readlines())

[b'\x80\x04\x95\x82\x01\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\r\n', b'superBlock\x94\x93\x94)\x81\x94}\x94(\x8c\x0bmagicNumber\x94KZ\x8c\r\n', b'freeBlocks\x94]\x94(h\x00\x8c\x05Inode\x94\x93\x94)\x81\x94}\x94(\x8c\x04name\x94\x8c\tim_a_file\x94\x8c\x05stats\x94\x8c\x02os\x94\x8c\x0bstat_result\x94\x93\x94(M\xb6\x81\x8a\x08J\x05\x03\x00\x00\x00\x84\x00J\x96\xce\xeaNK\x01K\x00K\x00M3\x02JQ\x9e+bJQ\x9e+bJ\xedC(bt\x94}\x94(\x8c\x08st_atime\x94GA\xd8\x8a\xe7\x94JW\xb7\x8c\x08st_mtime\x94GA\xd8\x8a\xe7\x94JW\xb7\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x08t\xcaG\\0i\xdb\x16\x8c\x0bst_mtime_ns\x94\x8a\x08t\xcaG\\0i\xdb\x16\x8c\x0bst_ctime_ns\x94\x8a\x08\x18\x08-qT\xa1\xda\x16\x8c\x12st_file_attributes\x94K \x8c\x0est_reparse_tag\x94K\x00u\x86\x94R\x94ubC\x14hello this is a test\x94eub.\xd7x\x8c\x08st_mtime\x94GA\xd8\x8a\xe7\x92"\xd7x\x8c\x08st_ctime\x94GA\xd8\x8a\x10\xfbs\x8cP\x8c\x0bst_atime_ns\x94\x8a\x080\xb7\xa7Z.i\xdb\x16\x8c\x0bst_mtime_ns\x94\x8

In [125]:
def tfs_stat(fd):
    for entry in dynamicResourceTable:
        if entry.fd == fd:
            fileIndex = entry.blockIndex
    for file in fileSystems:
        # found mounted file system now we get a single byte and write to buffer
        if file.mounted == 1:
            # readBlock(FD, blockIndex, buffer) 
            inode = file.superBlock.freeBlocks[0]
            print(inode.stats)

In [126]:
fd = tfs_open("im_a_file")
tfs_stat(fd)

os.stat_result(st_mode=33206, st_ino=37154696926004554, st_dev=1324011158, st_nlink=1, st_uid=0, st_gid=0, st_size=563, st_atime=1647025745, st_mtime=1647025745, st_ctime=1646805997)
